### Tratamento dos dados

In [2]:
import pandas as pd

In [ ]:
df_account = pd.read_csv("ACCOUNT.csv")
df_responsavel = pd.read_csv("RESPONSAVEL.csv", encoding="latin1")
df_transaction = pd.read_csv("TRANSACTION.csv")

Remoção de colunas "unnamed" da tabela TRANSACTION

In [3]:
df_transaction = df_transaction.loc[:, ~df_transaction.columns.str.startswith('Unnamed:')]

Prepara os DataFrames para análise ao garantir que as colunas selecionadas estejam em um formato numérico adequado, limpando caracteres indesejados e convertendo valores textuais em números.

In [6]:
def clean_and_convert(df, cols, replace_dict=None):
    for col in cols:
        if col in df.columns:
            if replace_dict:
                for pattern, repl in replace_dict.items():
                    df[col] = df[col].replace(pattern, repl, regex=True)
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

df_account = clean_and_convert(df_account, ['AUC'])
df_transaction = clean_and_convert(df_transaction, ['value'], {'[,]' : ''})
df_responsavel = clean_and_convert(df_responsavel, ['max_account_load', 'transaction_target', 'transaction_C_target'], {'[,]' : '', 'R\$': ''})


### Pergunta 1: Verificar se a quantidade de contas por responsável respeita os valores de "max_account_load"

Primeiro, conta o número de contas por responsável e, em seguida, compara esse número com o limite máximo. Por fim, exibe os responsáveis que estão dentro do limite e aqueles que excedem o limite.

In [7]:
contas_por_responsavel = df_account.groupby('responsavel_id').size().reset_index(name='quantidade_contas')
df_verificacao = contas_por_responsavel.merge(df_responsavel[['responsavel_id', 'max_account_load']], on='responsavel_id')
df_verificacao['dentro_do_limite'] = df_verificacao['quantidade_contas'] <= df_verificacao['max_account_load']

print("Resultados da verificação:")
print(df_verificacao)

df_excedentes = df_verificacao[~df_verificacao['dentro_do_limite']]
print("\nResponsáveis que excedem o limite:")
print(df_excedentes)

Resultados da verificação:
  responsavel_id  quantidade_contas  max_account_load  dentro_do_limite
0         000000                 95               NaN             False
1         0001AG                  9              15.0              True
2         0001DM                 16              15.0             False
3         0001JT                 16              15.0             False
4         0001LL                 20              15.0             False
5         0001LM                 15              15.0              True
6         0001RP                 12              15.0              True
7         0001TR                 17              15.0             False

Responsáveis que excedem o limite:
  responsavel_id  quantidade_contas  max_account_load  dentro_do_limite
0         000000                 95               NaN             False
2         0001DM                 16              15.0             False
3         0001JT                 16              15.0             False
4

### Pergunta 2: Calcular o valor transacionado por responsável

Calcula o valor total transacionado por cada responsável, associando as transações aos responsáveis e somando os valores. Após o cálculo, os valores são formatados como valores monetários em reais BRL e combinados com os nomes dos responsáveis. O resultado é exibido para cada responsável com o valor total transacionado.

In [8]:
df_transaction.rename(columns={'transaction_value': 'value'}, inplace=True)
df_transacoes_contas = df_transaction.merge(df_account[['client_id', 'responsavel_id']], on='client_id')
valor_transacionado_por_responsavel = df_transacoes_contas.groupby('responsavel_id')['value'].sum().reset_index(name='valor_total_transacionado')
valor_transacionado_por_responsavel['valor_total_transacionado'] = valor_transacionado_por_responsavel['valor_total_transacionado'].apply(lambda x: f'R${x:,.2f}')
df_resultado = valor_transacionado_por_responsavel.merge(df_responsavel[['responsavel_id', 'name']], on='responsavel_id')

print("Valor total transacionado por responsável:")
print(df_resultado)

Valor total transacionado por responsável:
  responsavel_id valor_total_transacionado     name
0         000000            R$6,543,508.00  DIGITAL
1         0001AG              R$504,709.00  Antonyo
2         0001DM              R$800,798.00   Daniel
3         0001JT            R$1,481,962.00     João
4         0001LL            R$1,220,359.00    Lucas
5         0001LM              R$922,451.00      Liz
6         0001RP              R$728,256.00   Rafael
7         0001TR              R$895,436.00    Tomas


### Pergunta 3: Levando em consideração a coluna "kind", calcule os valores creditados, debitados  e o valor liquido por responsável

Mapeia os tipos de transações para facilitar a distinção entre creditado e debitado. Em seguida, junta os dados de contas e transações, calcula os totais para cada tipo de transação e o valor líquido. Finalmente, adiciona os nomes dos responsáveis aos resultados e exibe essas informações.

In [9]:
df_transaction['kind'] = df_transaction['kind'].map({'C': 'credited', 'D': 'debited'})
merged_df = pd.merge(df_account, df_transaction, on='client_id', how='left')
summary = merged_df.groupby(['responsavel_id', 'kind'])['value'].sum().unstack(fill_value=0)
summary['net_value'] = summary.get('credited', 0) - summary.get('debited', 0)
result = summary.reset_index()
result = pd.merge(result, df_responsavel[['responsavel_id', 'name']], on='responsavel_id', how='left')

print("Valores creditados, debitados e valor líquido por responsável:")
print(result)

Valores creditados, debitados e valor líquido por responsável:
  responsavel_id   credited    debited  net_value     name
0         000000  3697270.0  2846238.0   851032.0  DIGITAL
1         0001AG   232632.0   272077.0   -39445.0  Antonyo
2         0001DM   355244.0   445554.0   -90310.0   Daniel
3         0001JT   328346.0  1153616.0  -825270.0     João
4         0001LL   495913.0   724446.0  -228533.0    Lucas
5         0001LM   787756.0   134695.0   653061.0      Liz
6         0001RP   290080.0   438176.0  -148096.0   Rafael
7         0001TR   537088.0   358348.0   178740.0    Tomas


### Pergunta 4: Demonstre quais responsáveis atingiram os objetivos de "transaction_target" e "transaction_C_target"


Primeiramente, soma os valores das transações e integra esses dados com os objetivos. Em seguida, verifica se cada responsável alcançou os objetivos de transação e crédito, e se ambos foram atingidos simultaneamente. O total de responsáveis que atingiram ambos os objetivos é exibido junto com os valores correspondentes. Também são apresentados detalhes formatados para esses responsáveis. Finalmente, são mostrados os totais para os responsáveis que não atingiram nenhum dos objetivos, incluindo valores transacionados e creditados.

In [39]:
summary['total_transacted'] = summary.sum(axis=1)
result = pd.merge(summary.reset_index(), df_responsavel[['responsavel_id', 'name', 'transaction_target', 'transaction_C_target']], on='responsavel_id', how='left')
result['target_achieved'] = result['total_transacted'] >= result['transaction_target']
result['credit_target_achieved'] = result.get('credited', 0) >= result['transaction_C_target']

result['both_targets_achieved'] = result['target_achieved'] & result['credit_target_achieved']
achieved_both_targets = result[result['both_targets_achieved']]

total_achieved_both = len(achieved_both_targets)
total_transacted_both = achieved_both_targets['total_transacted'].sum()
total_credited_both = achieved_both_targets['credited'].sum()

print(f"Total de responsáveis que atingiram ambos os objetivos: {total_achieved_both}")
print(f"Valor total transacionado por responsáveis que atingiram ambos os objetivos: R${total_transacted_both:,.2f}")
print(f"Valor total creditado por responsáveis que atingiram ambos os objetivos: R${total_credited_both:,.2f}")

if not achieved_both_targets.empty:
    print("\nDetalhes dos responsáveis que atingiram ambos os objetivos:")

    columns_to_display = [
        'responsavel_id', 'name', 'total_transacted',
        'credited', 'transaction_target',
        'transaction_C_target', 'target_achieved', 'credit_target_achieved'
    ]

    achieved_both_targets_display = achieved_both_targets[columns_to_display].copy()
    achieved_both_targets_display['total_transacted'] = achieved_both_targets_display['total_transacted'].apply(lambda x: f'R${x:,.2f}')
    achieved_both_targets_display['credited'] = achieved_both_targets_display['credited'].apply(lambda x: f'R${x:,.2f}')
    achieved_both_targets_display['transaction_target'] = achieved_both_targets_display['transaction_target'].apply(lambda x: f'R${x:,.2f}')
    achieved_both_targets_display['transaction_C_target'] = achieved_both_targets_display['transaction_C_target'].apply(lambda x: f'R${x:,.2f}')

    achieved_both_targets_display = achieved_both_targets_display.sort_values(by='name')

    print(achieved_both_targets_display.to_string(index=False))

not_achieved_targets = result[~result['both_targets_achieved']]

total_not_achieved = len(not_achieved_targets)
total_transacted_not_achieved = not_achieved_targets['total_transacted'].sum()
total_credited_not_achieved = not_achieved_targets['credited'].sum()

print(f"\nTotal de responsáveis que não atingiram nenhum objetivo: {total_not_achieved}")
print(f"Valor total transacionado por responsáveis que não atingiram nenhum objetivo: R${total_transacted_not_achieved:,.2f}")
print(f"Valor total creditado por responsáveis que não atingiram nenhum objetivo: R${total_credited_not_achieved:,.2f}")

if not not_achieved_targets.empty:
    print("\nDetalhes dos responsáveis que não atingiram nenhum dos objetivos:")

    columns_to_display_not_achieved = [
        'responsavel_id', 'name', 'total_transacted',
        'credited', 'transaction_target',
        'transaction_C_target', 'target_achieved', 'credit_target_achieved'
    ]

    not_achieved_targets_display = not_achieved_targets[columns_to_display_not_achieved].copy()
    not_achieved_targets_display['total_transacted'] = not_achieved_targets_display['total_transacted'].apply(lambda x: f'R${x:,.2f}')
    not_achieved_targets_display['credited'] = not_achieved_targets_display['credited'].apply(lambda x: f'R${x:,.2f}')
    not_achieved_targets_display['transaction_target'] = not_achieved_targets_display['transaction_target'].apply(lambda x: f'R${x:,.2f}')
    not_achieved_targets_display['transaction_C_target'] = not_achieved_targets_display['transaction_C_target'].apply(lambda x: f'R${x:,.2f}')

    not_achieved_targets_display = not_achieved_targets_display.sort_values(by='name')

    print(not_achieved_targets_display.to_string(index=False))


Total de responsáveis que atingiram ambos os objetivos: 6
Valor total transacionado por responsáveis que atingiram ambos os objetivos: R$210,854,978.00
Valor total creditado por responsáveis que atingiram ambos os objetivos: R$6,201,617.00

Detalhes dos responsáveis que atingiram ambos os objetivos:
responsavel_id    name total_transacted       credited transaction_target transaction_C_target  target_achieved  credit_target_achieved
        000000 DIGITAL R$125,707,180.00 R$3,697,270.00       R$950,000.00       R$2,100,000.00             True                    True
        0001DM  Daniel  R$12,078,296.00   R$355,244.00       R$157,500.00         R$350,000.00             True                    True
        0001JT    João  R$11,163,764.00   R$328,346.00        R$99,000.00         R$220,000.00             True                    True
        0001LM     Liz  R$26,783,704.00   R$787,756.00       R$139,500.00         R$310,000.00             True                    True
        0001LL   Lu

### Pergunta 5: Com base no AUC do inicio do mês e nas transações realizadas calcule o AUC das contas dos clientes no dia 30/06/2024

Primeiro, as datas nas colunas são convertidas para o tipo datetime. Em seguida, são filtradas as transações realizadas em junho de 2024 e agregadas por cliente e tipo (creditado ou debitado). A seguir, obtém-se o AUC inicial para o dia 1º de junho e mescla esses dados com os totais das transações. O saldo final de AUC é calculado adicionando o valor creditado e subtraindo o valor debitado do AUC inicial. São então filtrados apenas os clientes com valores não nulos. O resumo simplificado exibe o total de clientes, o valor total creditado, debitado e o AUC final para o dia 30/06/2024.

In [22]:
df_account['date'] = pd.to_datetime(df_account['date'], format='%m/%d/%Y')
df_transaction['date'] = pd.to_datetime(df_transaction['date'], format='%m/%d/%Y')

df_transaction_june = df_transaction[(df_transaction['date'].dt.month == 6) & (df_transaction['date'].dt.year == 2024)]

transaction_totals = df_transaction_june.groupby(['client_id', 'kind'])['value'].sum().unstack(fill_value=0)
transaction_totals.columns = ['debited', 'credited']  # Renomear colunas para clareza

initial_auc = df_account[df_account['date'].dt.day == 1][['client_id', 'AUC']].rename(columns={'AUC': 'initial_auc'})

result = initial_auc.merge(transaction_totals, on='client_id', how='left').fillna(0)

result['auc_final'] = result['initial_auc'] + result['credited'] - result['debited']

result_filtered = result[(result['initial_auc'] != 0) | (result['credited'] != 0) | (result['debited'] != 0)]

total_clients = result_filtered.shape[0]
total_credited = result_filtered['credited'].sum()
total_debited = result_filtered['debited'].sum()
total_auc_final = result_filtered['auc_final'].sum()

print(f"Total de clientes com AUC no dia 30/06/2024: {total_clients}")
print(f"Valor total creditado: R${total_credited:,.2f}")
print(f"Valor total debitado: R${total_debited:,.2f}")
print(f"Valor total AUC final: R${total_auc_final:,.2f}")


Total de clientes com AUC no dia 30/06/2024: 114
Valor total creditado: R$6,373,150.00
Valor total debitado: R$6,724,329.00
Valor total AUC final: R$-351,179.00
